In [1]:
import boa 
from utils import deploy_infra, deploy_pool, get_balances
from IPython.display import display, HTML

## **Table of Content**

1. [Infrastrucutre and Pool Setup](#infrastrucutre-and-pool-setup)
2. [Exchanging Tokens](#exchanging-tokens)
3. [get_dy](#get_dyi-uint256-j-uint256-dx-uint256---uint256)
4. [get_dx](#get_dxi-uint256-j-uint256-dy-uint256---uint256)
5. [fee_calc](#5-fee_calc)

### **1. Infrastrucutre and Pool Setup**

Before we can exchange tokens, we need to deploy the twocrypto-ng infrastructure and deploy a pool. Afterwards, we grant approval to the pool and add some initial liquidity.

In [2]:
# deploying infra for twocrypto-ng pools
infra = deploy_infra()
factory = infra["factory"]
trader = infra["trader"]
coin_a = infra["coin_a"]
coin_b = infra["coin_b"]
fee_receiver = infra["fee_receiver"]

# deploy pool
pool = deploy_pool(factory, coin_a, coin_b)
pool = infra["impl"].at(pool)

/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/ast/folding.py:153: UserWarning: ZERO_ADDRESS is deprecated. Please use `empty(address)` instead.
  warnings.warn(f"{name} is deprecated. Please use `{replacement}` instead.")
/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/semantics/types/subscriptable.py:115: UserWarning: Use of large arrays can be unsafe!
  warnings.warn("Use of large arrays can be unsafe!")


Before we can exchange tokens, we need to give the contract approval and add some initial liquidity.

In [3]:
# grant max approval and add some initial liquidity

with boa.env.prank(trader):
    for coin in [coin_a, coin_b]:
        coin.approve(pool, 2**256-1)

with boa.env.prank(trader):
    pool.add_liquidity([10**21, 10**21], 0)

print("Pool Balance CoinA:", pool.balances(0) / 1e18)
print("Pool Balance CoinB:", pool.balances(1) / 1e18)

Pool Balance CoinA: 1000.0
Pool Balance CoinB: 1000.0


### **2. Exchanging Tokens**

- #### `exchange(i: uint256, j: uint256, dx: uint256, min_dy: uint256, receiver: address = msg.sender) -> uint256:`

    Function to exchange `dx` amount of coin `i` for coin `j` and receive a minimum amount of `min_dy`. Charged fee at current states is `Pool.fee()` and is taken in token `j`.

    Returns: Amount of tokens at index `j` received (`uint256`).

    Emits: `TokenExchange`

    | Input       | Type      | Description                                                |
    | ----------- | --------- | ---------------------------------------------------------- |
    | `i`         | `uint256` | Index value for the input coin.                            |
    | `j`         | `uint256` | Index value for the output coin.                           |
    | `dx`        | `uint256` | Amount of input coin being swapped in.                     |
    | `min_dy`    | `uint256` | Minimum amount of output coin to receive.                  |
    | `receiver`  | `address` | Address to send output coin to. Defaults to `msg.sender`.  |

Index values of the coins can be fetched by `Pool.coins(n)`.

NOTE: `min_dy` should not be set to zero. This will allow MEV Bots to frontrun your transactions, resulting in receiving much less tokens than you should.


In [5]:
i = 0
j = 1
dx = 10**19
min_dy = 0
receiver = trader

before_trader_bal_a = coin_a.balanceOf(trader)
before_trader_bal_b = coin_b.balanceOf(trader)
before_pool_bal_a = pool.balances(0)
before_pool_bal_b = pool.balances(1)

# now we are exchanging 10 (10^19) coin_a for coin_b.
with boa.env.prank(trader):
    pool.exchange(i, j, dx, min_dy, receiver)


after_trader_bal_a = coin_a.balanceOf(trader)
after_trader_bal_b = coin_b.balanceOf(trader)
after_pool_bal_a = pool.balances(0)
after_pool_bal_b = pool.balances(1)

print(f"trader balance a: {(after_trader_bal_a - before_trader_bal_a) / 1e18}\n"
      f"trader balance b: {(after_trader_bal_b - before_trader_bal_b) / 1e18}\n"
      f"pool balance a: {(after_pool_bal_a - before_pool_bal_a) / 1e18}\n"
      f"pool balance b: {(after_pool_bal_b - before_pool_bal_b) / 1e18}")

trader balance a: -10.0
trader balance b: 9.956824290884265
pool balance a: 10.0
pool balance b: -9.956824290884265


As we can see, the pool now has 10 (10^19) additional coins of a while the trader has the same amount less. When exchanging, the tokens are transfered into the pool. 

On the contrary, the pool balance of coin b as decreased the same amount the traders balance of coin b increased.

### **3. `get_dy`**

- #### `get_dy(i: uint256, j: uint256, dx: uint256) -> uint256:`

    Getter for the received amount of coin `j` for swapping in `dx` amount of coin `i`. This method includes fees.

    Returns: Exact amount of output `j` tokens (`uint256`).

    | Input | Type      | Description               |
    | ----- | --------- | ------------------------- |
    | `i`   | `uint256` | Index of input token.     |
    | `j`   | `uint256` | Index of output token.    |
    | `dx`  | `uint256` | Amount of input tokens.   |

In [6]:
# how many tokens `j` do we receive when swapping in 10**19 of token `j`?
i = 0
j = 1
dx = 10**19

pool.get_dy(i, j, dx)

9855125262825180481

### **4. `get_dx`**

- #### `get_dx(i: uint256, j: uint256, dy: uint256) -> uint256:`

    Getter for the required amount of coin `i` to input for swapping out `dy` amount of token `j`.

    Returns: Amount of coins received (`uint256`).

    | Input | Type      | Description               |
    | ----- | --------- | ------------------------- |
    | `i`   | `uint256` | Index of input token.     |
    | `j`   | `uint256` | Index of output token.    |
    | `dy`  | `uint256` | Amount of output tokens.  |

In [7]:
# how many tokens `i` do we need to put in to receive 10**19 of token `j`?
i = 0
j = 1
dy = 10**19

pool.get_dx(i, j, dy)

10148424258456324461

### **5. `fee_calc`**

- #### `fee_calc(xp: uint256[N_COINS]) -> uint256:`

    Getter for the charged exchange fee by the pool at the current state.

    Returns: Fee value (`uint256`).

    | Input | Type               | Description                                      |
    | ----- | ------------------ | ------------------------------------------------ |
    | `xp`  | `uint256[N_COINS]` | Pool balances multiplied by the coin precisions. |

In [21]:
# get pool balances
balance_coinA = pool.balances(0)
balance_coinB = pool.balances(1)

# get coin precisions
precision_coinA = pool.precisions()[0]
precision_coinB = pool.precisions()[1]

# calculate xp
xp_a = balance_coinA * precision_coinA
xp_b = balance_coinB * precision_coinB

# calculate fee at current state
fee = pool.fee()
calc_fee = pool.fee_calc([xp_a, xp_b])

calc_fee, fee

(31740073, 31740073)

Precision of the fee is `10^10`. 

31740073 = 0.0031740073 = 0.31740073%.

`fee_calc(xp)` at the current pool state returns the same value as `fee()`.

---

*Quick crash course on cryptoswap fee values:*

1. `mid_fee`: Fee when the pool is perfectly balanced (lowest possible fee).
2. `out_fee`: Fee when the pool is completely imbalanced (highest possible fee).
3. `fee_gamma`: Parameter which determines the speed at which the fee increases when the pool becomes imbalanced.


In [22]:
mid_fee = pool.mid_fee()
out_fee = pool.out_fee()
fee_gamma = pool.fee_gamma()

mid_fee, out_fee, fee_gamma

(26000000, 45000000, 230000000000000)